# End to end Function Calling example


#### Pydantic is a data validation and settings management lib using Python type annotations =. It allows for the creation of data models through which you can enforce type hints at runtime, and probides a simple way to convert loosely typed data into stricter typed data. 

- Syntax:

In [2]:
import os
import openai
from typing import List
from pydantic import BaseModel, Field

In [4]:
# Normal python:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email
        
        
# Pydantic way:
class pUser(BaseModel):
    name: str
    age: int
    email: str

class Class(BaseModel):
    students: List[pUser]

In [5]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

In [11]:
openai_api_key = ""
open_ai_api_base = ""

openai.api_base = open_ai_api_base
openai.api_key = openai_api_key

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['OPENAI_BASE_URL'] = open_ai_api_base

# Pydantic to OpenAI: function definition conversion

OBS: Docstrings are a mandatory element to those functions

In [6]:
class WeatherSearch(BaseModel):
    
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [7]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [8]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [9]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

In [12]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI()
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}})

In [13]:
model_with_function = model.bind(functions=[weather_function])

In [14]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}})